In [2]:
import numpy as np
import os, sys
from datetime import datetime
import requests
from requests.exceptions import HTTPError
%load_ext autoreload
%autoreload 2

In [3]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [4]:
def get_transformation_matrix(animal):
    try:
        url = f'https://activebrainatlas.ucsd.edu/activebrainatlas/rotation/{animal}/manual/2'
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        transformation_matrix = response.json()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

    return transformation_matrix


In [6]:
prep_id = 'DK52'

In [7]:
transformation = get_transformation_matrix(prep_id)
r = transformation['rotation']
t = transformation['translation']

In [ ]:
r

In [8]:
# person_id = 2 = beth
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    # bili, do your transformation on x,y,section here
    row_dict[structure] = [row.x, row.y, row.section]


In [9]:
def brain_to_atlas_transform(brain_coord, r, t):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates. 
    """
    atlas_coord = brain_coord @ r + t 
    return atlas_coord


In [10]:
person_id = 28 # bili, this is your database ID
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    #print(k, structure, x, y, section)
    x,y,section = brain_to_atlas_transform((x,y,section), r, t)
    print(x,y,section)
    continue
    com = CenterOfMass(prep_id=prep_id, structure=structure, x=x, y=y, section=section,
                   created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned')
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()



NameError: name 'brain_to_atlas' is not defined